# 분류작업

In [4]:
import pandas as pd
df = pd.read_csv('pol_news.csv')
ngram = pd.read_csv('news_ngram_updown.csv')


C:\Users\15Z970-GA5BK\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
ngram_copy = ngram.copy()
token_copy = df.copy()
ngram_copy['token'] = token_copy['token']
added_df = ngram_copy.drop(['Unnamed: 0', 'news_media', 'text', 'title', 'url', 'price'], axis = 1)
added_df['ngrams'] = added_df['ngrams'].apply(lambda x: x.replace('[', '').replace(']', '').replace("'", ''))

# 뭘 만든거지.. 허허 각 row에 같이 있는 ngram과 token을 동시에 합치는 방법을 구상해야함

In [7]:

def classify_pos_tokens(dataframe):
    a = 0
    for up_down in dataframe['up_down']:
        if up_down == '상승':
            up_ngram = []
            for ngram in dataframe[dataframe['up_down'] == '상승']['ngrams']:

                split = ngram.split(',')
                count = 0
                if len(split) > 1:
                    for count in range(len(split)):
                        up_ngram.append(split[count])
                    a += 1
                    if a % 10000 == 0:
                        print(a)
            up_token = []
            for token in dataframe[dataframe['up_down'] == '상승']['token']:
                split = token.split(',')
                count = 0
                if len(split) > 1:
                    for count in range(len(split)):
                        up_ngram.extend(split[count])
                    a += 1
                    if a % 10000 == 0:
                        print(a)
            return up_ngram + up_token
        
def classify_neg_tokens(dataframe):
    a= 0
    for up_down in dataframe['up_down']:
        if up_down == '하락':
            down_ngram = []
            for ngram in dataframe[dataframe['up_down'] == '하락']['ngrams']:
                split = ngram.split(',')
                count = 0
                if len(split) > 1:
                    for count in range(len(split)):
                        down_ngram.extend(split[count])
                    a += 1
                    if a % 10000 == 0:
                        print(a)
            down_token = []
            for token in dataframe[dataframe['up_down'] == '하락']['token']:
                split = token.split(',')
                count = 0
                if len(split) > 1:
                    for count in range(len(split)):
                        up_ngram.expand(split[count])
                    a += 1
                    if a % 10000 == 0:
                        print(a)
            return down_ngram + down_token

        else:
            pass
        
added_df['ngram_token'] = 0
added_df['ngrams'] = added_df['ngrams'].fillna("")
added_df['ngram_token'] = classify_pos_tokens(added_df)


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000


AttributeError: 'float' object has no attribute 'split'

In [ ]:
print(len(up_ngram), len(down_ngram))

# 병합작업

In [ ]:

up_ngram_test = up_ngram
down_ngram_test =down_ngram

In [ ]:
up_ngram_test.extend(up_result)
down_ngram_test.extend(down_result)

In [ ]:
ngrams = pd.DataFrame()

In [ ]:
ngrams['하락'] = pd.Series(down_ngram_test)
ngrams['상승'] = pd.Series(up_ngram_test)

## 각 극성 value_Counts()

In [ ]:
up_counts = ngrams['상승'].value_counts()
down_counts = ngrams['하락'].value_counts()

In [ ]:
ngrams['up_ngrams'] = 0
ngrams['up_ngrams'] = up_counts.to_frame().reset_index()
ngrams['up_ngrams_counts'] = pd.Series(up_counts.values)

In [ ]:
ngrams['down_ngrams'] = 0
ngrams['down_ngrams'] = down_counts.to_frame().reset_index()
ngrams['down_ngrams_counts'] = pd.Series(down_counts.values)

# 15개 미만 잘라내기

In [ ]:
ngrams['상승'] = pd.Series(up_counts.loc[up_counts > 15])
ngrams['하락'] = pd.Series(down_counts.loc[down_counts > 15])

In [ ]:
ngrams.rename_axis(index = 'ngram_id')
ngrams.fillna(1)

In [ ]:
up_sum = ngrams['상승'].values.sum()
down_sum = ngrams['하락'].values.sum()
ngrams['up_score'] = ngrams['상승'].apply(lambda x: x/up_sum)
ngrams['down_score'] = ngrams['하락'].apply(lambda x: x/down_sum)

# polarity score

In [ ]:
def f(x):
    return x[2]/x[3]
ngrams['polarity_score'] = ngrams.apply(lambda x: f(x), axis = 1)

# 범위 설정

In [ ]:
ngrams[(ngrams['polarity_score'] < 0.73) | (ngrams['polarity_score'] > 1.3)]